<a href="https://colab.research.google.com/github/Swapn2/PyTorch/blob/main/cnn_fashion_mnist_pytorch_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [36]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [37]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [38]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df.shape

(60000, 785)

In [40]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
X_train = X_train/255.0
X_test = X_test/255.0

In [43]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [44]:
train_dataset = CustomDataset(X_train, y_train)

In [45]:
test_dataset = CustomDataset(X_test, y_test)

In [46]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [47]:
class MyNN(nn.Module):
  def __init__(self, input_feature):
    super().__init__()

    self.features = nn.Sequential(
        nn.Conv2d(input_feature , 32 , kernel_size = 3, padding = 'same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2, stride = 2),
        nn.Conv2d(32 , 64 , kernel_size = 3, padding = 'same'),
        nn.ReLU(),
        nn.BatchNorm2d(64), # Changed from 32 to 64
        nn.MaxPool2d(kernel_size=2, stride = 2)
    )
    self.classification = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7, 128),
        nn.ReLU(),
        nn.Dropout(p = .4),

        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(p = .4),
        nn.Linear(64,10)
    )

  def forward(self,x):
    x = self.features(x)
    x = self.classification(x)

    return x

In [51]:
learning_rate = 0.01
epochs = 20

In [52]:
model = MyNN(1)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [53]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 0.6519195455337564
Epoch: 2 , Loss: 0.38297766718765097
Epoch: 3 , Loss: 0.32165951825678346
Epoch: 4 , Loss: 0.2901204180791974
Epoch: 5 , Loss: 0.26611925680935383
Epoch: 6 , Loss: 0.24810139880826076
Epoch: 7 , Loss: 0.22987215274572373
Epoch: 8 , Loss: 0.21855225983510415
Epoch: 9 , Loss: 0.2021199192032218
Epoch: 10 , Loss: 0.19526157931859295
Epoch: 11 , Loss: 0.1852404932671537
Epoch: 12 , Loss: 0.1768188184176882
Epoch: 13 , Loss: 0.16674735090881587
Epoch: 14 , Loss: 0.15499260830770556
Epoch: 15 , Loss: 0.14861920318379998
Epoch: 16 , Loss: 0.13749736135794471
Epoch: 17 , Loss: 0.13252634447502593
Epoch: 18 , Loss: 0.12559577082144097
Epoch: 19 , Loss: 0.12109301908034831
Epoch: 20 , Loss: 0.1133494928337944


In [54]:
model.eval()

MyNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [55]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9199166666666667


In [56]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.9777916666666666
